In [1]:
import mat_properties as prop

# Считывание рефпропа
RP = prop.init_REFPROP(r"C:\Program Files (x86)\REFPROP")
gasmix = "Nitrogen*Oxygen*CO2*Water*Argon"
fractiongas = (
    0.715784461314205,
    0.0114726415414296,
    0.143332198252282,
    0.116991486761544,
    0.0124192121305392
)


water = prop.Materials_prop(
    "Water",
    [1.0, 0, 0, 0, 0],
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    prop.REFPROP_s_q,
    RP=RP,
)


Methane = prop.Materials_prop(
    "METHANE",
    [1.0, 0, 0, 0, 0],
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    prop.REFPROP_s_q,
    RP=RP,
)


fractionwaterMethane = (0.848859488304273, 0.151140511695727, 0, 0, 0)
waterMethanemix = "Water*METHANE"

waterMethane = prop.Materials_prop(
    waterMethanemix,
    fractionwaterMethane,
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    prop.REFPROP_s_q,
    RP=RP,
)

gas = prop.Materials_prop(
    gasmix,
    fractiongas,
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    prop.REFPROP_s_q,
    RP=RP,
)


G1 = 47.56
T11 = 509
P11 = 7.845

dT=15
dTmin=5
Tdec = 20

P2 = 2
Pdr1 = 2
Pdr2 = 0.8


H11 = water.p_t(P11, T11)['h']
Q11 = water.p_h(P11, H11)['Q']

Q12 = 0
P12 = P11
H12 = water.p_q(P12, Q12)['h']
T12 = water.p_q(P12, Q12)['T']

P13 = Pdr1
H13 = H12
T13 = water.p_h(P13, H13)['T']
Q13 = water.p_h(P13, H13)['Q']

P24 = P2
T24 = T11-dT
H24 = water.p_t(P24, T24)['h']

T23 = T13-dTmin
P23 = P24
H23 = water.p_t(P23, T23)['h']

G2 = G1 * ((H11-H12)/(H24-H23))

Q14 = 0
P14 = P13
H14 = water.p_q(P14, Q14)['h']
T14 = water.p_q(P14, Q14)['T']

P15 = Pdr2
H15 = H14
T15 = water.p_h(P15, H15)['T']
Q15 = water.p_h(P15, H15)['Q']

T16 = T15 - Tdec
P16 = P15
H16 = water.p_t(P16, T16)['h']

P22 = P2
H22 = H23 - (G1*(H13-H14)/G2)
T22 = water.p_h(P22, H22)['T']

P21 = P2
H21 = H22 - (G1*(H15-H16)/G2)
T21 = water.p_h(P21, H21)['T']


# print(G1*(H15-H16))
# print(G2*(H22-H21))


print(H21)

print(water.p_h(2, H21)['T'])

print(water.p_h(2, 64.89281964700012)['T'])
print(water.p_t(2, 15)['h'])

# print(T11,T12,T13,T14,T15,T16)
# print(T21,T22,T23,T24)

64.39389106631376
-71.6826934614677
-71.68985563044959
64.89281964700012
